Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/automated-machine-learning/production-deploy-to-aks-gpu/production-deploy-to-aks-gpu.png)

# Deploying a web service hosted on NVIDIA Triton to Azure Kubernetes Service (AKS)
This notebook shows the steps for deploying a service with [NVIDIA Triton Inferencing Server](https://developer.nvidia.com/nvidia-triton-inference-server): registering a model, creating an image, provisioning a cluster (one time action), and deploying a service to it. 
We then test and delete the service, image and model.

In [ ]:
import azureml.core
print(azureml.core.VERSION)

# Get workspace
Load existing workspace from the config file info.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Download the model

Prior to registering the model, you should have a model in one of the [supported formats](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#framework-model-definition) by Triton. This cell will download a [pretrained ONNX densenet](https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx) model.

In [ ]:
import os
import requests
import shutil
import tarfile
import tempfile

from io import BytesIO

model_url = 'https://contentmamluswest001.blob.core.windows.net/content/14b2744cf8d6418c87ffddc3f3127242/9502630827244d60a1214f250e3bbca7/08aed7327d694b8dbaee2c97b8d0fcba/densenet121-1.2.onnx'

target_file = os.path.join('models', 'triton', 'densenet_onnx', '1', 'model.onnx')

if not os.path.exists(target_file):
    response = requests.get(model_url)
    open(target_file, 'wb').write(response.content)

# Register the model
Register an existing trained model, add description and tags.

** Note: ** Under `model_path` there must be a sub-directory named `triton`, which has the structure of a Triton [Model Repository](https://docs.nvidia.com/deeplearning/triton-inference-server/user-guide/docs/model_repository.html#repository-layout)

In [ ]:
from azureml.core.model import Model

model = Model.register(model_path="models", # This points to the local directory to upload.
                       model_name="densenet_onnx", # This is the name the model is registered as.
                       tags={'area': "Image classification", 'type': "classification"},
                       description="Image classification trained on Imagenet Dataset",
                       workspace=ws)

print(model.name, model.description, model.version)

# Provision the AKS Cluster
This is a one time setup. You can reuse this cluster for multiple deployments after it has been created. If you delete the cluster or the resource group that contains it, then you would have to recreate it.

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your GPU cluster
gpu_cluster_name = "aks-gpu-cluster"

# Verify that cluster does not exist already
try:
    gpu_cluster = ComputeTarget(workspace=ws, name=gpu_cluster_name)
    print("Found existing gpu cluster")
except ComputeTargetException:
    print("Creating new gpu-cluster")
    
    # Specify the configuration for the new cluster
    compute_config = AksCompute.provisioning_configuration(cluster_purpose=AksCompute.ClusterPurpose.DEV_TEST,
                                                           agent_count=1)
                                                           #vm_size="Standard_NV6")
    # Create the cluster with the specified name and configuration
    gpu_cluster = ComputeTarget.create(ws, gpu_cluster_name, compute_config)

    # Wait for the cluster to complete, show the output log
    gpu_cluster.wait_for_completion(show_output=True)

# Deploy the model as a web service to AKS

First create a scoring script

** Note: ** Triton server listens to a fixed local port. You may choose to use the Triton python client to talk to it, while keeping the flexibility of pre-/post- processing.

In [ ]:
%%writefile score.py
import argparse
import numpy as np
from PIL import Image
import sys
from functools import partial
import os
import io

import tritonhttpclient
from tritonclientutils import triton_to_np_dtype
from tritonclientutils import InferenceServerException

from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse

def preprocess(img, scaling):
    """
    Pre-process an image to meet the size, type and format
    requirements specified by the parameters.
    """
    # np.set_printoptions(threshold='nan')
    c = 3
    h = 224
    w = 224
    format = "FORMAT_NCHW"

    
    if c == 1:
        sample_img = img.convert('L')
    else:
        sample_img = img.convert('RGB')

    resized_img = sample_img.resize((w, h), Image.BILINEAR)
    resized = np.array(resized_img)
    if resized.ndim == 2:
        resized = resized[:, :, np.newaxis]

    npdtype = triton_to_np_dtype(dtype)
    typed = resized.astype(npdtype)

    if scaling == 'INCEPTION':
        scaled = (typed / 128) - 1
    elif scaling == 'VGG':
        if c == 1:
            scaled = typed - np.asarray((128,), dtype=npdtype)
        else:
            scaled = typed - np.asarray((123, 117, 104), dtype=npdtype)
    else:
        scaled = typed

    # Swap to CHW if necessary
    if format == "FORMAT_NCHW":
        ordered = np.transpose(scaled, (2, 0, 1))
    else:
        ordered = scaled

    # Channels are in RGB order. Currently model configuration data
    # doesn't provide any information as to other channel orderings
    # (like BGR) so we just assume RGB.
    return ordered


def postprocess(results, output_name, batch_size, batching):
    """
    Post-process results to show classifications.
    """

    output_array = results.as_numpy(output_name)
    if len(output_array) != batch_size:
        raise Exception("expected {} results, got {}".format(
            batch_size, len(output_array)))

    # Include special handling for non-batching models
    output = ""
    for results in output_array:
        if not batching:
            results = [results]
        for result in results:
            if output_array.dtype.type == np.bytes_:
                cls = "".join(chr(x) for x in result).split(':')
            else:
                cls = result.split(':')
            output += "    {} ({}) = {}".format(cls[0], cls[1], cls[2])

    return output

trition_client = None
_url = "localhost:8000"
_model = "densenet_onnx"
_scaling = "INCEPTION"

def init():
    global triton_client, max_batch_size, input_name, output_name, dtype
    
    print("Connect to Triton")
    triton_client = tritonhttpclient.InferenceServerClient(_url)
    
    max_batch_size = 0
    input_name = "data_0"
    output_name = "fc6_1"
    dtype = "FP32"

@rawhttp
def run(request):
    if request.method == 'POST':
        
        reqBody = request.get_data(False)
        img = Image.open(io.BytesIO(reqBody))
        
        image_data = preprocess(img, _scaling)
        
        input = tritonhttpclient.InferInput(input_name, image_data.shape, dtype)
        input.set_data_from_numpy(image_data, binary_data=True)
        output = tritonhttpclient.InferRequestedOutput(output_name, binary_data=True, class_count=1)
    
        res = triton_client.infer(_model,
                                [input],
                                request_id="0",
                                outputs=[output])

        result = postprocess(res, output_name, 1, max_batch_size > 0)

        return AMLResponse(result, 200)
    else:
        return AMLResponse("bad request", 500)


In [ ]:
triton_client_url = 'https://github.com/NVIDIA/triton-inference-server/releases/download/v2.1.0/v2.1.0_ubuntu1804.clients.tar.gz'

client_filename = 'tritonclientutils-2.1.0-py3-none-any.whl'
clientutils_filename = 'tritonhttpclient-2.1.0-py3-none-any.whl'
target_folder = 'python_client'

if not os.path.exists(target_folder):
    response = requests.get(triton_client_url)
    archive = tarfile.open(fileobj=BytesIO(response.content))
    with tempfile.TemporaryDirectory() as temp_folder:
        archive.extractall(temp_folder)
        os.mkdir(target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', client_filename), target_folder)
        shutil.copy(os.path.join(temp_folder, 'python', clientutils_filename), target_folder)


Now create the deployment configuration objects and deploy the model as a webservice.

** Note: ** If you use the Triton python client, also include them in the `Environment`.

In [ ]:
# Set the web service configuration (using default here)
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.environment import Environment

env = Environment("deploytocloudenv")
conda_dep = CondaDependencies("env.yml")
client_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, client_filename), exist_ok=True)
clientutils_whl_url = Environment.add_private_pip_wheel(workspace=ws, file_path = os.path.join(target_folder, clientutils_filename), exist_ok=True)
conda_dep.add_pip_package(client_whl_url)
conda_dep.add_pip_package(clientutils_whl_url)
env.python.conda_dependencies = conda_dep

# TODO: replace with MCR image name
#env.docker.base_image = DEFAULT_GPU_IMAGE
env.docker.base_image = "delliwscentrec595b96.azurecr.io/azureml-tritonserver:20.07-py3"
env.docker.base_image_registry.address = "delliwscentrec595b96.azurecr.io"
env.docker.base_image_registry.username = "delliwscentrec595b96"
env.docker.base_image_registry.password = "eOlKQlx+tFa+J3Ml5a86fOHf8i=hi/ec"

# Optionally specify a worker count to leverage the capability of concurrency and server-side batching from Triton
# env.environment_variables = {"WORKER_COUNT":"128"}

inference_config = InferenceConfig(entry_script="score.py", environment=env)
aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

# # Enable token auth and disable (key) auth on the webservice
# aks_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4, token_auth_enabled=True, auth_enabled=False)

In [ ]:
%%time
aks_service_name ='gpu-densenet-onnx'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=gpu_cluster)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

In [ ]:
aks_service.delete()

# Test the web service
We test the web sevice by passing the test images content.

In [ ]:
%%time
import requests

# if (key) auth is enabled, fetch keys and include in the request
key1, key2 = aks_service.get_keys()

headers = {'Content-Type':'application/octet-stream', 'Authorization': 'Bearer ' + key1}

# # if token auth is enabled, fetch token and include in the request
# access_token, fetch_after = aks_service.get_token()
# headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + access_token}

test_sample = open('car.jpg', 'rb').read()
resp = requests.post(aks_service.scoring_uri, test_sample, headers=headers)

# Clean up
Delete the service, image, model and compute target

In [ ]:
%%time
aks_service.delete()
model.delete()
gpu_cluster.delete()
